#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 2:   word2vec + SVM + Evaluation

### 100 points [6% of your final grade]

### Due: Tuesday, February 26, 2019 by 11:59pm

*Goals of this homework:* Understand word2vec-like term embeddings,  explore real-world challenges with SVM-based classifiers, understand and implement several evaluation metrics.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, my homework submission would be something like `555001234_hw2.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Term embeddings + SVM (80 points)

### Dataset


For this homework, we will still play with Yelp reviews from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). As in Homework 1, you'll see that each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. Additionally, this time, we also offer you the "label". If `label=1`, it means that this review is `Food-relevant`. If `label=0`, it means that this review is `Food-irrelevant`. Similarly, we have already done some basic preprocessing on the reviews, so you can just tokenize each review using whitespace.

There are about 40,000 reviews in total, in which about 20,000 reviews are "Food-irrelevant". We split the review data into two sets. *review_train.json* is the training set. *review_test.json* is the testing set. 

In [60]:
import json
import numpy as np
from sklearn.svm import SVC
import gensim
import math


In [111]:
# Please load the dataset
# Your code below
all_train_data = []
all_test_data = []
with open("review_train.json") as f:
    for line in f:
        all_train_data.append(json.loads(line))
        
with open("review_test.json") as f:
    for line in f:
        all_test_data.append(json.loads(line))



###  Pre-trained term embeddings

To save your time, you can make use of  pre-trained term embeddings. In this homework, we are using one of the great pre-trained models from [GloVe](https://nlp.stanford.edu/projects/glove/) based on 2 billion tweets. GloVe is quite similar to word2vec. Unzip the *glove.6B.50d.txt.zip* file and run the code below. You will be able to load the term embeddings model, with which each word can be represented with a 50-dimension vector.

In [112]:
# reload the pre-trained term embeddings

with open("glove.6B.50d.txt", "rb") as lines:
    model = {line.split()[0].decode(encoding="utf-8", errors="strict"): np.array(list(map(float, line.split()[1:])))
           for line in lines}

Now, you have a vector representation for each word. First, we use the simple (arithmetic) **mean** of these vectors of words in a review to represent the review. *Note: Just ignore those words which are not in the corpus of this pre-trained model.*

In [113]:
# Please figure out the vector representation for each review in the training data and testing data.
# Your code below
# get the vector for each review in trainning data and correponding labels
def vector_for_review(review, model):
    res = np.zeros(50)
    count = 0;
    for word in review:
        if word in model:
            res += model.get(word)
            count += 1;
    res /= count;
    return res;

train_data_vectors = np.zeros((len(all_train_data), 50))
train_data_labels = np.zeros(len(all_train_data))
train_data_reviews = []
i = 0
for train_data in all_train_data:
    review = train_data.get("review").split()
    train_data["review_score"] = vector_for_review(review, model)
    train_data_vectors[i] = train_data["review_score"]
    train_data_labels[i] = train_data["label"]
    train_data_reviews.append(train_data.get("review"))
    i += 1


In [114]:
# get the vector for each review in test data and correponding labels
test_data_vectors = np.zeros((len(all_test_data), 50))
test_data_labels = np.zeros(len(all_test_data))
test_data_reviews = []
i = 0
for test_data in all_test_data:
    review = test_data.get("review").split()
    test_data["review_score"] = vector_for_review(review, model)
    test_data_vectors[i] = test_data["review_score"]
    test_data_labels[i] = test_data["label"]
    test_data_reviews.append(test_data.get("review"))
    i += 1

### SVM

With the vector representations you get for each review, please train an SVM model to predict whether a given review is food-relevant or not. **You do not need to implement any classifier from scratch. You may use scikit-learn's built-in capabilities.** You can only train your model with reviews in *review_train.json*.

In [115]:
# SVM model training
# Your code here
# train the classifier using SVM
clf_trained = SVC(gamma="scale")
clf_trained.fit(train_data_vectors, train_data_labels)



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Your goal is to predict whether a given review is food-relevant or not. Please report the overall accuracy, precision and recall of your model on the **testing data**. You should **implement the functions for accuracy, precision, and recall**.

In [116]:
# Your code here
def getParameters(test_result, test_labels):
    # given the predicted results for test data and true label, return TP, TN, FP, FN
    TP = TN = FP = FN = 0
    for i in range(len(test_labels)):
        if test_labels[i] == 0 and test_result[i] == 0:
            TN += 1
        elif test_labels[i] == 0 and test_result[i] == 1:
            FP += 1
        elif test_labels[i] == 1 and test_result[i] == 1:
            TP += 1
        elif test_labels[i] == 1 and test_result[i] == 0:
            FN += 1
    return TP, TN, FP, FN
            
    
    
def getResult (TP, TN, FP, FN):
    # given TP, TN, FP, FN, return accurancy, precision and recall
    accurancy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return accurancy, precision, recall
    
    
test_result = clf_trained.predict(test_data_vectors)
TP, TN, FP, FN = getParameters(test_result, test_data_labels)
accurancy, precision, recall = getResult(TP, TN, FP, FN)



In [134]:
print("Result using pre-trained term embeddings with SVM classifier\n")
print("accurancy: ", accurancy)
print("precision: ", precision)
print("recall: ", recall)



Result using pre-trained term embeddings with SVM classifier

accurancy:  0.9211409395973155
precision:  0.9185482522160896
recall:  0.9238015138772078


### Document-based embeddings

Instead of taking the mean of term embeddings, you can directly train a **doc2vec** model for paragraph or document embeddings. You can refer to the paper [Distributed Representations of Sentences and Documents](https://arxiv.org/pdf/1405.4053v2.pdf) for more details. And in this homework, you can make use of the implementation in [gensim](https://radimrehurek.com/gensim/models/doc2vec.html).

Now, you need to:
* Train a doc2vec model based on all reviews you have (training + testing sets).
* Use the embeddings from your doc2vec model to represent each review and train a new SVM model.
* Report the overall accuracy, precision and recall of your model on the testing data.

In [118]:
# Train a doc2vec
# Your code here

# train a doc2vec model using all reviews(training + testing)
data_reviews = train_data_reviews + test_data_reviews
documents = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(doc), [i]) for i, doc in enumerate(data_reviews)]
model = gensim.models.doc2vec.Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4, epochs=40)
%time model = gensim.models.doc2vec.Doc2Vec(documents, vector_size=50, window=2, min_count=1, workers=4, epochs=40)


CPU times: user 5min 33s, sys: 25.6 s, total: 5min 59s
Wall time: 2min 33s


In [119]:
# get the corpus after simple_preprocess for both train and test data
def read_corpus(reviews):
    for i, line in enumerate(reviews):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

train_corpus = list(read_corpus(train_data_reviews))
test_corpus = list(read_corpus(test_data_reviews))



In [120]:
def get_vector (corpus, model):
    # given corpus, using the model we trained to get corresponding embedding vectors
    vec = np.zeros((len(corpus), 50))
    i = 0
    for corpu in corpus:
        vec[i] = model.infer_vector(corpu.words)
        i += 1
    return vec
    

In [121]:
# Train a SVM
# Your code here
train_data_doc2vec_vec = get_vector(train_corpus, model)
clf_doc2vec = SVC(gamma="scale")
clf_doc2vec.fit(train_data_doc2vec_vec, train_data_labels)



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [122]:
# Predict the test set using the SVM classifier we just trained, and assess the model via accurancy, precision and recall
test_data_doc2vec_vec = get_vector(test_corpus, model)
test_doc2vec_result = clf_doc2vec.predict(test_data_doc2vec_vec)
TP, TN, FP, FN = getParameters(test_doc2vec_result, test_data_labels)
accurancy, precision, recall = getResult(TP, TN, FP, FN)

In [133]:
# Report the performance
# Your code here
print("Result using self-trained doc2vec model for document embeddings\n")
print("accurancy: ", accurancy)
print("precision: ", precision)
print("recall: ", recall)


Result using self-trained doc2vec model for document embeddings

accurancy:  0.9211409395973155
precision:  0.9185482522160896
recall:  0.9238015138772078


What do you observe? How different are your results for the term-based average approach vs. the doc2vec approach? Why do you think this is?

*provide a brief (1-2 paragraph) discussion based on these questions.*

### Can you do better?

Finally, see if you can do better than either the word- or doc- based embeddings approach for classification. You may explore new features, new classifiers, etc. Whatever you like. Just provide your code and a justification.

In [ ]:
# your code here
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree')
nbrs.fit

# Part 2: NDCG (20 points)

You calculated the recall and precision in Part 1 and now you get a chance to implement NDCG. 

Assume that Amy searches for "food-relevant" reviews in the **testing set** on two search engines `A` and `B`. Since the ground-truth labels for the reviews are unknown to A and B, they need to make a prediction for each review and then return a ranked list of results based on their probabilities. The results from A are in *search_result_A.json*, and the results from B are in *search_result_B.json*. Each line contains the id of a review and its corresponding ranking.

You can check their labels in *review_test.json* while calculating the NDCG scores. If a review is "food-relevant", the relevance score is 1. Otherwise, the relevance score is 0.

In [124]:
# read the given json files
search_result_A = []
search_result_B = []
with open("search_result_A.json") as f:
    for line in f:
        search_result_A.append(json.loads(line))

with open("search_result_B.json") as f:
    for line in f:
        search_result_B.append(json.loads(line))



In [125]:
def DCG_score(rel, i):
    # calculate the DCG Score, given rel, i
    return rel / (math.log2(i + 1));

def id_rel(datas):
    # mapping the id with label
    id_rel = {}
    for data in datas:
        id_rel[data.get("id")] = data.get("label")
    return id_rel


In [126]:
# preprocess the data, given a id we know the relevant 
test_id2rel = id_rel(all_test_data)

In [127]:
# NDCG for search_result_A.json
# Your code here
DCG_A = 0
for result in search_result_A:
    # calculate the DCG first
    cur_id = result.get("id")
    rel = test_id2rel.get(cur_id)
    DCG_A += DCG_score(rel, result.get("rank"))

IDCG_A = 0
new_rank = 1
for result in search_result_A:
    # then calculate the IDCG
    cur_id = result.get("id")
    if test_id2rel.get(cur_id) == 1:
        IDCG_A += DCG_score(1, new_rank)
        new_rank += 1

NDCG_A = DCG_A / IDCG_A
print(DCG_A, IDCG_A, NDCG_A)



505.3979278728301 521.4752244965487 0.96916958684041


In [128]:
# NDCG for search_result_B.json
# Your code here
DCG_B = 0
for result in search_result_B:
    cur_id = result.get("id")
    rel = test_id2rel.get(cur_id)
    DCG_B += DCG_score(rel, result.get("rank"))

IDCG_B = 0
new_rank = 1
for result in search_result_B:
    cur_id = result.get("id")
    if test_id2rel.get(cur_id) == 1:
        IDCG_B += DCG_score(1, new_rank)
        new_rank += 1

NDCG_B = DCG_B / IDCG_B
print(DCG_B, IDCG_B, NDCG_B)


121.34577387849328 121.68560167059829 0.9972073294831962


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*